In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from mabwiser.mab import MAB, LearningPolicy

# Carregar Dados

In [2]:
df_contx = pd.read_csv('../dataset/processed/df_contexto.csv')
df_contx.drop(columns=['last_click'], inplace=True)
df_contx.fillna(value={'recency_last_click':999_999_999_999}, inplace=True)
df_contx.sort_values(['sent_at'],ascending=True,inplace=True)
# df_contx['data'] = pd.to_datetime(df_contx['sent_at']).dt.date

cols_not_contx = ['id', 'message_id', 'campaign_id', 'client_id', 'sent_at']
# df_contx = df_contx[0:10_000]
df_contx.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_16352\859839591.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_contx = pd.read_csv('../dataset/processed/df_contexto.csv')


,id,message_id,campaign_id,client_id,sent_at,sending_time,recency_last_click,message_type_bulk,message_type_transactional,message_type_trigger,...,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,behavioral_segment_engajador,behavioral_segment_ocasional,behavioral_segment_silencioso
283875,3521581,2a627fd2-0ced-462d-9709-df7a42ac1a3e,27,1515915625468226660,2021-04-30 00:02:04,0,1.000000e+12,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5093474,3523598,a28cc53b-0ab9-4589-ac7b-e00334c622ae,27,1515915625504954588,2021-04-30 00:03:09,0,1.000000e+12,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9189980,3524551,dd7cf019-41d2-4bb1-bde7-bc009b769ee4,27,1515915625629231547,2021-04-30 00:03:43,0,1.000000e+12,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6113566,3527937,24045071-6242-4550-af74-b147b7ebbdeb,32,1515915625552553920,2021-04-30 00:04:27,0,1.000000e+12,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
243497,3525182,cc036cd8-2d63-4f76-87da-34ea37dd33a1,28,1515915625468202824,2021-04-30 00:05:11,0,1.000000e+12,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
df_action = pd.read_csv('../dataset/processed/df_action.csv')
df_action.drop(columns=['message_type', 'channel'],inplace=True)
df_action.head()

,id,message_id,campaign_id,client_id,sent_at,period_day,action
0,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,1515915625467997301,2021-04-30 09:00:18,manha,1
1,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,1515915625467997301,2021-04-30 13:06:09,tarde,3
2,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,1515915625467997301,2021-04-30 16:07:05,tarde,3
3,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,1515915625467997301,2021-05-04 08:27:03,manha,1
4,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,1515915625467997301,2021-05-04 09:08:17,manha,1


In [4]:
df_reward = pd.read_csv('../dataset/processed/df_reward.csv')
df_reward.drop(columns=['message_type', 'channel'],inplace=True)
df_reward.head()

,id,message_id,campaign_id,client_id,sent_at,reward
0,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,1515915625467997301,2021-04-30 09:00:18,0
1,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,1515915625467997301,2021-04-30 13:06:09,0
2,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,1515915625467997301,2021-04-30 16:07:05,0
3,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,1515915625467997301,2021-05-04 08:27:03,0
4,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,1515915625467997301,2021-05-04 09:08:17,0


# Simulação

In [5]:
def create_models_cmab_mab(context_matrix, action_list):
    n_features = context_matrix.shape[1]
    arms = sorted(set(action_list))

    mab_contextual = MAB(arms=arms, learning_policy=LearningPolicy.LinUCB(alpha=0.1))
    mab_nao_contextual = MAB(arms=arms, learning_policy=LearningPolicy.UCB1(alpha=0.1))

    mab_contextual.partial = True
    mab_nao_contextual.partial = True

    # Corrigir erro com dtype explícito
    mab_contextual.fit(decisions=[], rewards=[], contexts=np.empty((0, n_features)))
    mab_nao_contextual.fit(decisions=[], rewards=[])

    return mab_contextual, mab_nao_contextual


In [6]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

def predict_cmab_batch(contextos, true_actions, rewards, mab_contextual):
    preds = mab_contextual.predict(contexts=contextos)
    rewards_calc = [r if p == t else 0 for p, t, r in zip(preds, true_actions, rewards)]
    return preds, rewards_calc

def predict_mab_batch(true_actions, rewards, mab_nao_contextual):
    preds = [mab_nao_contextual.predict() for _ in true_actions]
    rewards_calc = [r if p == t else 0 for p, t, r in zip(preds, true_actions, rewards)]
    return preds, rewards_calc

def simulation(df_contx, df_action, df_reward, slot_size=10):
    # Chaves comuns para merge
    chaves = ['id', 'message_id', 'campaign_id', 'client_id', 'sent_at']

    # Merge vetorizado para unir contexto, ação e recompensa
    df = df_contx.merge(df_action, on=chaves, how='inner') \
                 .merge(df_reward, on=chaves, how='inner')

    df
    
    # Modelos
    cols_not_contx = ['id', 'message_id', 'campaign_id', 'client_id', 'sent_at', 'action', 'reward', 'period_day']
    cols_contexto = [col for col in df.columns if col not in cols_not_contx]

    X_contexto = df[cols_contexto]
    actions = df['action'].tolist()

    mab_contextual, mab_nao_contextual = create_models_cmab_mab(X_contexto, actions)

    # Histórico
    cumulative_contextual = 0
    cumulative_nao_contextual = 0
    x_data, y_cmab, y_mab = [], [], []

    # Colunas de contexto
    cols_not_contx = chaves + ['action', 'reward', 'period_day']
    cols_contexto = [col for col in df.columns if col not in cols_not_contx]

    n = df.shape[0]

    for i in range(0, n, slot_size):
        df_slot = df.iloc[i:i + slot_size]

        contextos = df_slot[cols_contexto].values.tolist()
        true_actions = df_slot['action'].tolist()
        rewards = df_slot['reward'].tolist()

        # --- CMAB ---
        preds_ctx, rewards_ctx = predict_cmab_batch(contextos, true_actions, rewards, mab_contextual)
        mab_contextual.fit(decisions=preds_ctx, rewards=rewards_ctx, contexts=contextos)
        cumulative_contextual += sum(rewards_ctx)
        # cumulative_contextual += sum([100 if r == 1 else -10 for r in rewards_ctx])

        # --- MAB ---
        preds_mab, rewards_mab = predict_mab_batch(true_actions, rewards, mab_nao_contextual)
        mab_nao_contextual.fit(decisions=preds_mab, rewards=rewards_mab)
        # cumulative_nao_contextual += rewards_mab
        cumulative_nao_contextual += sum(rewards_mab)
        # cumulative_nao_contextual += sum([100 if r == 1 else -10 for r in rewards_mab])

        # Gráfico
        x_data.append(i + slot_size)
        y_cmab.append(cumulative_contextual)
        y_mab.append(cumulative_nao_contextual)

        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(x_data, y_cmab, label="LinUCB (Contextual)")
        ax.plot(x_data, y_mab, label="UCB (Não-Contextual)", linestyle="--")
        ax.set_xlabel("Interações")
        ax.set_ylabel("Recompensa acumulada ($)")
        ax.set_title(f"CMAB ({cumulative_contextual}) vs MAB ({cumulative_nao_contextual}) - Interação {min(i+slot_size, n)}/{n}")
        ax.legend()
        ax.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:
simulation(df_contx, df_action, df_reward, slot_size=1000)